In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [130]:
X_train_fpath = '/Users/fengchi-luen/Desktop/機器學習/ml2019spring-hw2/X_train'
Y_train_fpath = '/Users/fengchi-luen/Desktop/機器學習/ml2019spring-hw2/Y_train'
X_test_fpath = '/Users/fengchi-luen/Desktop/機器學習/ml2019spring-hw2/X_test'

X_train = np.genfromtxt(X_train_fpath, delimiter=',', skip_header=1)
Y_train = np.genfromtxt(Y_train_fpath, delimiter=',', skip_header=1)
X_test = np.genfromtxt(X_test_fpath, delimiter=',', skip_header=1)

In [127]:
def normalization(X, column, train = True ,X_mean = None, X_std = None):
    if train:
        #先建立好mean與std
        X_mean = np.reshape(np.mean(X[:, column],0), (1, len(column)))
        X_std  = np.reshape(np.std(X[:, column],0), (1,len(column)))
        X[:, column] = np.divide(np.subtract(X[:, column],X_mean), X_std)
        return X, X_mean, X_std
    
    X[:, column] = np.divide(np.subtract(X[:, column],X_mean), X_std)
    
    return X

In [5]:
col = [0,1,3,4,5,7,10,12,25,26,27,28]
X_train, X_mean, X_std = normalization(X_train, col, train = True)
Y_train = Y_train.reshape(-1,1)

Probability Model

In [18]:
#先把class_0, class_1分出來
class_0_id = []
class_1_id = []
for i in range(Y_train.shape[0]):
    if Y_train[i] == 1:
        class_1_id.append(i)
    else:
        class_0_id.append(i)
class_0 = X_train[class_0_id]
class_1 = X_train[class_1_id]

In [24]:
class_0.shape[0]

24720

In [94]:
#計算mean與cov

#mean
mean_0 = np.mean(class_0, axis = 0).reshape([1,-1])
mean_1 = np.mean(class_1, axis = 0).reshape([1,-1])

#加權的cov
n = class_0.shape[1]
cov_0 = np.zeros((n,n))
cov_1 = np.zeros((n,n))

row0 = class_0.shape[0]
row1 = class_1.shape[0]

for i in range(class_0.shape[0]):
    cov_0 += np.dot((class_0[i] - mean_0).T, (class_0[i] - mean_0)) / row0
for i in range(class_1.shape[0]):
    cov_1 += np.dot((class_1[i] - mean_1).T, (class_1[i] - mean_1)) / row1
cov = (cov_0 * row1 + cov_1 * row2) / (row0 + row1)


In [124]:
#計算P(C｜x) = sigmoid(z) 的 z
from numpy.linalg import inv
w = ((mean_0 - mean_1)).dot( inv(cov) )
b = (-0.5) * np.dot(np.dot(mean_0,inv(cov)), mean_0.T) \
    + 0.5 * np.dot(np.dot(mean_1,inv(cov)), mean_1.T) \
    +np.log(row0 / row1)

In [129]:
X_test.shape

(16281, 14)

In [131]:
#計算所有答案
col = [0,1,3,4,5,7,10,12,25,26,27,28]
X_test = normalization(X_test, col, train = False ,X_mean = X_mean, X_std = X_std)
arr = []
for i in range(X_test.shape[0]):
    z = w.dot(X_test[i]) + b
    z *= (-1)
    z = 1 / (1 + np.exp(z))
    arr.append( np.clip(z, (1e-8), 1 - (1e-8)))

In [141]:
ans = []
for i in range(X_test.shape[0]):
    if arr[i] >= 0.5:
        ans.append(1)
    else:
        ans.append(0)